# Energy Load Forecasting — Power BI Prep
**Purpose:** Clean the smart-meter dataset, compute KPIs, and export tidy CSVs for a Power BI dashboard.

**Inputs:** `/mnt/data/transformed_smart_meter_data.csv`

**Outputs (saved to `/mnt/data/powerbi_exports`)**
- `fact_timeseries.csv` — long table of timestamped load by meter/location
- `kpis.csv` — single-row KPIs (loss ratio, average hourly load, peak demand, monthly consumption)
- `by_location.csv` — current KPIs by location
- `monthly_consumption.csv` — monthly sums per location
- `data_dictionary.md` — column descriptions

> **Note on Loss Ratio**: If your dataset has supply vs. billed/consumed columns, set the names in the config cell below. Otherwise the loss ratio will be `NaN` and you can fill it from another source.

In [3]:

# ==== 0. Imports & Config ====
import pandas as pd, numpy as np
from pathlib import Path

# --- File paths ---
RAW_PATH = Path("smart_meter_data.csv")    # change if needed
OUTDIR = Path(""); OUTDIR.mkdir(exist_ok=True)

# --- Column configuration ---
# Attempt to auto-detect common columns; override if needed.
TIMESTAMP_COL = None   # e.g., "timestamp"
LOAD_COL = None        # e.g., "kWh" or "load"
LOCATION_COL = None    # e.g., "location"
MSN_COL = None         # e.g., "msn" (meter serial number)

# If you have technical loss inputs, set these:
SUPPLY_ENERGY_COL = None   # e.g., "energy_input_kWh" (feeder input)
BILLED_ENERGY_COL = None   # e.g., "metered_energy_kWh" (sum of meters/billed)

# Granularity to standardize to (Power BI friendly)
RESAMPLE_RULE = "1H"   # hourly


In [4]:

# ==== 1. Load and inspect ====
df = pd.read_csv(RAW_PATH)
print("Columns:", list(df.columns))
df.head(3)


Columns: ['datetime', 'msn', 'location', 'daily_consumption_load', 'energy_supplied', 'energy_billed']


,datetime,msn,location,daily_consumption_load,energy_supplied,energy_billed
0,2021-01-01 00:00:00,ACE43B7D,Chennai,2.59,2.78,2.57
1,2021-01-01 00:15:00,ACE43B7D,Chennai,1.78,2.12,1.86
2,2021-01-01 00:30:00,ACE43B7D,Chennai,2.59,2.68,2.52


In [5]:

# ==== 2. Infer column names if not provided ====
def guess(colnames, keys):
    cand = [c for c in colnames if any(k in c.lower() for k in keys)]
    return cand[0] if cand else None

cols = list(df.columns)
TIMESTAMP_COL = TIMESTAMP_COL or guess(cols, ["time","date","timestamp"])
LOAD_COL      = LOAD_COL      or guess(cols, ["consum","kwh","load","usage","energy"])
LOCATION_COL  = LOCATION_COL  or guess(cols, ["loc","city","region","site","location"])
MSN_COL       = MSN_COL       or guess(cols, ["msn","meter","serial","meter_id","id"])

print("Detected TIMESTAMP_COL:", TIMESTAMP_COL)
print("Detected LOAD_COL     :", LOAD_COL)
print("Detected LOCATION_COL :", LOCATION_COL)
print("Detected MSN_COL      :", MSN_COL)

assert TIMESTAMP_COL is not None, "Could not detect timestamp column — set TIMESTAMP_COL"
assert LOAD_COL is not None, "Could not detect load/consumption column — set LOAD_COL"


Detected TIMESTAMP_COL: datetime
Detected LOAD_COL     : daily_consumption_load
Detected LOCATION_COL : location
Detected MSN_COL      : msn


In [6]:

# ==== 3. Parse timestamps, clean, and standardize to hourly ====
ts = pd.to_datetime(df[TIMESTAMP_COL], errors="coerce", utc=True).dropna()
df = df.loc[ts.index].copy()
df["timestamp"] = ts.dt.tz_convert("Asia/Kolkata")  # localize to IST; adjust if needed
df = df.sort_values("timestamp")

# Keep relevant columns
keep = ["timestamp", LOAD_COL]
if LOCATION_COL and LOCATION_COL in df.columns: keep.append(LOCATION_COL)
if MSN_COL and MSN_COL in df.columns: keep.append(MSN_COL)
df = df[keep].rename(columns={LOAD_COL:"load"}).copy()

# If there are multiple readings per hour, aggregate to sum (or mean for power)
df = df.set_index("timestamp")
df_hourly = df.groupby([pd.Grouper(freq=RESAMPLE_RULE)] + ([LOCATION_COL] if LOCATION_COL else []) + ([MSN_COL] if MSN_COL else [])).sum(numeric_only=True)
df_hourly["load"] = df_hourly["load"].interpolate(limit_direction="both")
df_hourly = df_hourly.reset_index()

df_hourly.head(3)


C:\Users\RRC\AppData\Local\Temp\ipykernel_31284\4201890951.py:15: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_hourly = df.groupby([pd.Grouper(freq=RESAMPLE_RULE)] + ([LOCATION_COL] if LOCATION_COL else []) + ([MSN_COL] if MSN_COL else [])).sum(numeric_only=True)


,timestamp,location,msn,load
0,2021-01-01 05:00:00+05:30,Bengaluru,0653DAA9,6.05
1,2021-01-01 05:00:00+05:30,Chennai,ACE43B7D,4.37
2,2021-01-01 05:00:00+05:30,Delhi,7F6ACD62,4.76


In [8]:

# ==== 4. Compute KPIs ====
# Average Hourly Load across dataset
avg_hourly_load = df_hourly["load"].mean()

# Peak Demand (max hourly load)
peak_demand = df_hourly["load"].max()

# Monthly Consumption (sum over the latest full calendar month across all meters)
df_hourly["month"] = df_hourly["timestamp"].dt.to_period("M")
latest_full_month = df_hourly["month"].max()
monthly_consumption = df_hourly.loc[df_hourly["month"] == latest_full_month, "load"].sum()

# Loss Ratio — needs supply vs billed columns; compute if available in raw df at original granularity.
if (SUPPLY_ENERGY_COL and SUPPLY_ENERGY_COL in df.columns) and (BILLED_ENERGY_COL and BILLED_ENERGY_COL in df.columns):
    # Aggregate to month for stability
    tmp = df[[SUPPLY_ENERGY_COL, BILLED_ENERGY_COL]].copy()
    tmp["timestamp"] = df.index
    tmp = tmp.resample("1M").sum(min_count=1)
    loss_ratio = float(1.0 - (tmp[BILLED_ENERGY_COL].sum() / max(1e-6, tmp[SUPPLY_ENERGY_COL].sum())))
else:
    loss_ratio = np.nan  # left blank unless configured

kpi_row = {
    "loss_ratio": loss_ratio,
    "average_hourly_load": avg_hourly_load,
    "peak_demand": peak_demand,
    "monthly_consumption": monthly_consumption,
    "latest_full_month": str(latest_full_month)
}
kpis = pd.DataFrame([kpi_row])
kpis


C:\Users\RRC\AppData\Local\Temp\ipykernel_31284\2064226066.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df_hourly["month"] = df_hourly["timestamp"].dt.to_period("M")


,loss_ratio,average_hourly_load,peak_demand,monthly_consumption,latest_full_month
0,NaN,10.170702,18.74,252.61,2024-01


In [ ]:

# ==== 5. By-location and monthly tables (for slicers/filters) ====
if LOCATION_COL:
    # Average Hourly Load by location
    avg_by_loc = df_hourly.groupby(LOCATION_COL, as_index=False)["load"].mean().rename(columns={"load":"average_hourly_load"})
    # Peak Demand by location
    peak_by_loc = df_hourly.groupby(LOCATION_COL, as_index=False)["load"].max().rename(columns={"load":"peak_demand"})
    # Latest month consumption by location
    latest_month = df_hourly["month"].max()
    month_mask = df_hourly["month"] == latest_month
    mon_by_loc = df_hourly.loc[month_mask].groupby(LOCATION_COL, as_index=False)["load"].sum().rename(columns={"load":"monthly_consumption"})
    by_location = avg_by_loc.merge(peak_by_loc, on=LOCATION_COL, how="outer").merge(mon_by_loc, on=LOCATION_COL, how="outer")
else:
    by_location = pd.DataFrame()

# Monthly sums per location (for charts)
if LOCATION_COL:
    monthly = (df_hourly
               .groupby([LOCATION_COL, "month"], as_index=False)
               .agg(monthly_consumption=("load","sum")))
else:
    monthly = (df_hourly
               .groupby(["month"], as_index=False)
               .agg(monthly_consumption=("load","sum")))

by_location.head(3) if not by_location.empty else monthly.head(3)


,location,average_hourly_load,peak_demand,monthly_consumption
0,Bengaluru,10.156018,18.48,47.86
1,Chennai,10.176937,18.64,56.04
2,Delhi,10.191449,18.49,53.09


In [10]:

# ==== 6. Export tidy CSVs for Power BI ====
fact = df_hourly[["timestamp","load"] + ([LOCATION_COL] if LOCATION_COL else []) + ([MSN_COL] if MSN_COL else [])].copy()
fact.to_csv(OUTDIR/"fact_timeseries.csv", index=False)
kpis.to_csv(OUTDIR/"kpis.csv", index=False)
if not by_location.empty:
    by_location.to_csv(OUTDIR/"by_location.csv", index=False)
monthly.to_csv(OUTDIR/"monthly_consumption.csv", index=False)

dict_txt = '''
# Data Dictionary
- fact_timeseries.csv
  - timestamp (datetime, IST)
  - load (numeric; hourly sum or mean depending on your raw data)
  - location (text, optional)
  - msn (text, optional)

- kpis.csv
  - loss_ratio (float; requires config)
  - average_hourly_load (float)
  - peak_demand (float)
  - monthly_consumption (float; latest full month)
  - latest_full_month (text; YYYY-MM)

- by_location.csv
  - average_hourly_load, peak_demand, monthly_consumption by location

- monthly_consumption.csv
  - monthly_consumption by month (and location if available)
'''
with open(OUTDIR/"data_dictionary.md","w") as f:
    f.write(dict_txt)

print("Exports written to:", OUTDIR)


Exports written to: .


In [4]:
import pandas as pd
df = pd.read_csv('smart_meter_data.csv')
df.sample(20)
df['location'].value_counts()


location
Chennai      105120
Delhi        105120
Bengaluru    105120
Kolkata      105120
Mumbai       105120
Name: count, dtype: int64

## DAX snippets (copy into Power BI)


```DAX
-- If you also imported ML outputs:
MAE :=
AVERAGEX(
    'test_predictions',
    ABS('test_predictions'[actual] - 'test_predictions'[predicted])
)

RMSE :=
VAR _mse =
    AVERAGEX(
        'test_predictions',
        POWER('test_predictions'[actual] - 'test_predictions'[predicted], 2)
    )
RETURN SQRT(_mse)

Average Hourly Load := AVERAGE('fact_timeseries'[load])
Peak Demand := MAX('fact_timeseries'[load])

Monthly Consumption :=
VAR _lastMonth = EOMONTH(TODAY(), -1)
RETURN
CALCULATE(
    SUM('fact_timeseries'[load]),
    DATESINPERIOD('fact_timeseries'[timestamp], _lastMonth, MONTH, 1)
)
```

## Build the Power BI page


1) **Get Data → Text/CSV**: import `fact_timeseries.csv`, `kpis.csv` (and `test_predictions.csv`, `next24h_forecast.csv` if using ML visuals).
2) **Data model**: Set `timestamp` to Date/Time. Disable Auto Date/Time if you prefer.
3) **Cards**: Create four cards bound to (a) `loss_ratio` (format as percentage), (b) `Average Hourly Load`, (c) `Peak Demand`, (d) `Monthly Consumption`.
4) **Trend sparkline**: Area/Line chart with Axis=`timestamp`, Values=`load`. Add a **Relative Date slicer** (Last 30/90 days). Turn off gridlines and legends for a clean tile.
5) **Filter table**: Add a Table visual with `location` and `msn` (if present). Use it as a selector.
6) **Optional visuals**: 
   - Column chart of `monthly_consumption` by `month` (and `location` stacked).
   - Error charts using `test_predictions.csv` with MAE/RMSE measures.
7) **Formatting**: Increase card fonts; align tiles to match your Superset layout; apply a dark theme if desired.
8) **Publish**: Save PBIX → Publish to Power BI Service → Configure **Scheduled Refresh** if the CSVs live in OneDrive/SharePoint.
